In [1]:
import random
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import copy
from collections import defaultdict
from sklearn.metrics import precision_recall_curve, auc

In [2]:
def compute_true_false(preds, targets, dist_thresh):
    """
    compute the number of true and false predictions with the Hungarian algorithm

    Args:
        preds (array): coordinates of n predicted loops, array of shape (n, 2)
        targets (array): coordinates of m ground truth loop annotations, array of shape (m, 2)
        dist_thresh (float): threshold of the distance between two coordinates for them to be matched
    
    Returns:
        int, int, int, float, float, float: true positive, false positive, false negative, precision, recall, F1-score
    """

    if len(preds) == 0:
        # return all zeros if no prediction was made
        return 0, 0, 0, 0, 0, 0

    dist_matrix = cdist(preds, targets, 'euclidean')  # shape: (pred size, target size) 

    candidate_matrix = np.where(dist_matrix <= dist_thresh, 1, 0)
    # Candidate(i, j) = 1 means prediction i is close enough to targete j
    
    # the problem of uniquely assigning targets with predictions can be solved by the Hungarian algorithm
    # first, reverse of the candidate matrix to a cost matrix, Cost(i, j) = 0 iff Candidate(i, j) = 1, inf otherwise
    # we didn't use negative costs to fit the standard setting of the assignment problem
    # math.inf will cause problems with linear_sum_assignment(), using a large number instead
    cost_matrix = np.where(candidate_matrix == 1, 0, 10**10)

    # pad the cost matrix into a square matrix
    max_dim = max(cost_matrix.shape)
    pad_rows = max_dim - cost_matrix.shape[0]
    pad_cols = max_dim - cost_matrix.shape[1]
    cost_matrix = np.pad(cost_matrix, ((0, pad_rows), (0, pad_cols)), 'constant', constant_values=10**10)
    
    # print('[debug] compute_true_false(): cost matrix shape (afer padding):', cost_matrix.shape)

    # fit the Hungarian algorithm to find the optimal solution
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # the solution is represented as index pairs, no repetitive rows or columns are used
    # but it might choose an index pair from the padding region or choose an entry with large cost (sub-optimal)
    # as a result, we need to manually remove those assignments

    index_pairs = list(zip(row_ind, col_ind))  # list of two-element tuples
    # print(f'[debug] compute_true_false(): before post-processing, {len(index_pairs)} assignments')

    final_pairs = copy.deepcopy(index_pairs)

    for pair in index_pairs:
        # to be a valid assignment, the cost must be 0 (i.e., the entry must be 1 in the candidate matrix)
        # and the indcies must not be in the padding region
        if cost_matrix[pair] != 0 or pair[0] >= candidate_matrix.shape[0] or pair[1] >= candidate_matrix.shape[1]:
            final_pairs.remove(pair)

    # print(f'[debug] compute_true_false(): after post-processing, {len(final_pairs)} assignments')

    # the true positive is then the number of remaining assignments
    tp = len(final_pairs)

    # total predictions - true positives (unassigned predictions)
    fp = preds.shape[0] - tp

    # total targets - true positives (unassigned targets)
    fn = targets.shape[0] - tp

    try:
        precision = tp / preds.shape[0]
        recall = tp / targets.shape[0]
        f1score = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        # in case TP=0, then all 3 metrics should be zero
        precision, recall, f1score = 0, 0, 0

    return tp, fp, fn, precision, recall, f1score



In [3]:
def read_loops(loop_path):
    loops = []
    score_id = 6
    if 'chromosight' in loop_path:
        score_id = 10
    elif 'train' in loop_path:
        score_id = 0
    elif 'hiccups' in loop_path:
        score_id = 16
    
    with open(loop_path, 'r') as loop_file:
        for line in loop_file:
            if line.strip('\n').split('\t')[0] == 'chrom1':
                continue
            if '#' in line.strip('\n').split('\t')[0] :
                continue
            
            line_list = line.strip('\n').split('\t')

            loop_info = line_list[:6]
            
            if score_id !=0:
                if 'hicexplorer' in loop_path or 'hiccups' in loop_path:
                    loop_score = 1- float(line_list[score_id])
                    loop_info.append(loop_score)
                    loops.append(loop_info)
                else:
                    loop_score = float(line_list[score_id])
                    loop_info.append(loop_score)
                    loops.append(loop_info)
            else:
                loops.append(loop_info)          
    return loops

In [4]:
threshold_list = np.arange(0, 1.1, 0.1)

In [5]:
cell_type = 'gm12878'

In [ ]:
gt_file = '/Dataset/HiC/hic/loop_train/ctcf_{}.bedpe'.format(cell_type)



gt_loops = read_loops(gt_file)
print(len(gt_loops))

threshold_list = np.arange(0, 1.1, 0.1)

PR_Dic = {
    'Chromosight':[],
    'HiCExplorer':[],
    'YOLOOP':[],
    'Peakachu':[],
    'HICCUPS':[]
}




for benchmark in ['HICCUPS','Chromosight','HiCExplorer','YOLOOP','Peakachu']:

    print("method: {}".format(benchmark))
    if benchmark == 'YOLOOP':
        pred_file = 'yoloop_prediction/10kb/yoloop_pred_{}.bedpe'.format(cell_type)
        pred_loops = read_loops(pred_file)
    elif benchmark == 'HICCUPS':
        pred_file = 'benchmarks/hiccups/{}-hic_10kb/merged_loops.bedpe'.format(cell_type)
        pred_loops = read_loops(pred_file)
    elif benchmark == 'Chromosight':
        pred_file = 'benchmarks/chromosight/{}-hic_10kb/{}-hic_10kb.tsv'.format(cell_type,cell_type)
        pred_loops = read_loops(pred_file)
    elif benchmark == 'HiCExplorer':
        pred_file = 'benchmarks/hicexplorer/{}-hic_10kb/{}-hic_10kb.bedgraph'.format(cell_type,cell_type)
        pred_loops = read_loops(pred_file)
    elif benchmark == 'Peakachu':
        pred_loops = []
        for i in [1,9,14]:
            pred_file = 'benchmarks/peakachu/{}-hic_ctcf-chiapet_10kb/pool/chr{}.bedpe'.format(cell_type,i)
            chr_loops = read_loops(pred_file)
            pred_loops +=chr_loops
    
    
    # PR-Threshold
    for threshold in threshold_list:

        thresholded_pred = []

        for pred in pred_loops:
            score = float(pred[-1])
            if score < threshold:
                continue
            thresholded_pred.append(pred)
        print('Threshold : {}'.format(threshold))
        
        precision_list = []
        recall_list = []
                
        for target_chrom in ['chr1','chr9','chr14']:
            gt_list = []
            pred_list = []

            for pred_loop in tqdm(thresholded_pred):
                pred_chr = pred_loop[0]

                if 'chr' not in pred_loop[0]:
                    pred_chr = 'chr'+str(pred_loop[0])

                if pred_chr !=target_chrom:
                    continue
                x = int((int(pred_loop[1]) + int(pred_loop[2])) * 0.5)
                y = int((int(pred_loop[4]) + int(pred_loop[5])) * 0.5)
                pred_list.append([min(x, y), max(x, y)])

            for gt_loop in gt_loops:

                if gt_loop[0] !=target_chrom:
                    continue
                x = int((int(gt_loop[1]) + int(gt_loop[2])) * 0.5)
                y = int((int(gt_loop[4]) + int(gt_loop[5])) * 0.5)
                gt_list.append([min(x, y), max(x, y)])
    
        
            _, _, _, precision, recall, _ = compute_true_false( np.array(pred_list),np.array(gt_list), 10 * 10000)
            precision_list.append(precision)
            recall_list.append(recall)
        avg_precision = np.mean(precision)
        avg_recall = np.mean(recall)
        PR_Dic[benchmark].append([avg_precision,avg_recall])


55086
method: HICCUPS
Threshold : 0.0


100%|██████████| 8469/8469 [00:00<00:00, 1682927.97it/s]


Threshold : 0.1


100%|██████████| 8469/8469 [00:00<00:00, 1482597.80it/s]


Threshold : 0.2


...

100%|██████████| 50896/50896 [00:00<00:00, 1251125.55it/s]


Threshold : 0.4


100%|██████████| 50896/50896 [00:00<00:00, 1499120.75it/s]


Threshold : 0.5


100%|██████████| 50896/50896 [00:00<00:00, 411406.14it/s]


In [ ]:
PR_Dic

In [ ]:
# Calculate AUC

for method in PR_Dic.keys():
    precision_list = [PR_Dic[method][i][0] for i in range(len(PR_Dic[method]))]
    recall_list = [PR_Dic[method][i][1] for i in range(len(PR_Dic[method]))]
    
    pr_auc = auc(recall_list, precision_list)
    
    print("{}: AUC:{}".format(method,pr_auc))